# SQLMESH_explore


In [ ]:
import json
import duckdb
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


Agora que já baixamos os Materiais e colocamos no BD, vamos baixar as unidades que são usadas pelos materiais que baixamos.

In [1]:
import json
import duckdb
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
db = duckdb.connect('../db.db') 

In [7]:
s = db.execute("SELECT snapshots FROM sqlmesh._environments WHERE name='dev'").fetchall()
s

[('[{"name": "\\"db\\".\\"br_ibge__dtb\\".\\"territorio\\"", "change_category": 1, "fingerprint": {"data_hash": "3190608320", "metadata_hash": "2709304121", "parent_data_hash": "0", "parent_metadata_hash": "0"}, "previous_versions": [], "version": "3390729591", "physical_schema": "sqlmesh__br_ibge__dtb", "parents": [], "kind_name": "FULL", "node_type": "model"}, {"name": "\\"db\\".\\"br_mgi__siasg\\".\\"materiais\\"", "change_category": 1, "fingerprint": {"data_hash": "308998398", "metadata_hash": "2875864287", "parent_data_hash": "0", "parent_metadata_hash": "0"}, "previous_versions": [], "version": "871649237", "physical_schema": "sqlmesh__br_mgi__siasg", "parents": [], "kind_name": "FULL", "node_type": "model"}, {"name": "\\"db\\".\\"br_mgi__siasg\\".\\"compras\\"", "change_category": 1, "fingerprint": {"data_hash": "3430017474", "metadata_hash": "2109228832", "parent_data_hash": "0", "parent_metadata_hash": "0"}, "previous_versions": [{"fingerprint": {"data_hash": "2224546688", "me

In [8]:
json_result = [json.loads(row[0]) for row in s]

In [11]:
print(json.dumps(json_result, indent=4))

[
    [
        {
            "name": "\"db\".\"br_ibge__dtb\".\"territorio\"",
            "change_category": 1,
            "fingerprint": {
                "data_hash": "3190608320",
                "metadata_hash": "2709304121",
                "parent_data_hash": "0",
                "parent_metadata_hash": "0"
            },
            "previous_versions": [],
            "version": "3390729591",
            "physical_schema": "sqlmesh__br_ibge__dtb",
            "parents": [],
            "kind_name": "FULL",
            "node_type": "model"
        },
        {
            "name": "\"db\".\"br_mgi__siasg\".\"materiais\"",
            "change_category": 1,
            "fingerprint": {
                "data_hash": "308998398",
                "metadata_hash": "2875864287",
                "parent_data_hash": "0",
                "parent_metadata_hash": "0"
            },
            "previous_versions": [],
            "version": "871649237",
            "physical_schema": "sqlm

In [20]:
materiais = list(db.sql("select distinct codigoItem from br_mgi__siasg__dev.materiais where statusItem=true and codigoPdm in (select distinct * from read_csv('../seeds/pdms.csv', header=false, columns={'column0': 'BIGINT'})) order by codigoItem").df()['codigoItem'])
(materiais[:5], len(materiais))

([5410, 6467, 7838, 56235, 111333], 34698)

In [21]:
def fetch_and_save_despesas(codes):
    base_url = 'https://dadosabertos.compras.gov.br/modulo-pesquisa-preco/1_consultarMaterial' 
    
    def save_json(code, anopage):
        response = requests.get(f'{base_url}?codigoItemCatalogo={code}&pagina={page}').json()
        if len(response["resultado"])>0:
            if page==1: os.makedirs(f'../seeds/compras/codigoItemCatalogo={code}', exist_ok=True)
            with open(f'../seeds/compras/codigoItemCatalogo={code}/page_{page}.json', 'w') as f:
                json.dump(response["resultado"], f)
        return response
    
    for code in  progress_bar(codes, leave=False):
        response = save_json(code, 1)
        total_pages = response.get("totalPaginas", 0)
        if total_pages > 1:
            for page in range(2, total_pages + 1):
                save_json(code, page)

In [22]:
fetch_and_save(materiais)

In [ ]:
268236, 448982,273009, 

In [ ]:
import requests

url = "https://dadosabertos.compras.gov.br/modulo-pesquisa-preco/1_consultarMaterial?codigoItemCatalogo=407620&pagina=1"

payload = {}
headers = {
  'Accept': '*/*'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)